<a href="https://colab.research.google.com/github/yota-nagai/combinatorial-optimization-problems/blob/main/11_%E6%9C%89%E5%90%91%E5%B8%B0%E9%82%84%E9%A0%82%E7%82%B9%E9%9B%86%E5%90%88%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q amplify
from amplify import VariableGenerator, FixstarsClient, solve, Solver
from datetime import timedelta

client = FixstarsClient()
client.token = "******************"  #アクセストークンを入力
client.parameters.timeout = timedelta(milliseconds=2000)  # タイムアウトは 2000 ms


#################### 有向帰還頂点集合問題 ####################

N,M=map(int,input().split())  #頂点数、辺の数
edge=[]  #辺のリスト（有向グラフ）
for _ in range(M):
  u,v=map(int,input().split())
  u-=1
  v-=1
  edge.append([u,v])

#バイナリ変数の生成
gen=VariableGenerator()
y_v=gen.array("Binary",N)  #頂点vをそのまま残すか、取り除いて帰還頂点集合に含めるか
x_vi=gen.array("Binary",shape=(N,N+1))  #頂点vが高さiあるかどうか（残した頂点集合で）

#制約項①　残す各頂点に割り当てられる高さはただ一つ
f1=0
for v in range(N):
  cnt=0
  for i in range(1,N+1):
    cnt+=x_vi[v][i]
  f1+=(y_v[v]-cnt)**2

#制約項②　全ての辺が低い方から高い方へ向かわなければダメ（サイクル禁止）
f2=0
for u,v in edge:
  for i in range(N+1):
    for j in range(i,N+1):
      f2+=x_vi[u][i]*x_vi[v][j]

#目的関数項　取り除く（帰還頂点集合に入れる）頂点数を最小化したい
f3=0
for v in range(N):
  f3+=(1-y_v[v])

#ペナルティ係数の設定
A,B=map(int,input().split())
f=A*(f1+f2)+B*f3


#ソルバー実行、最適解の取得
result=solve(f,client)
objective=result.best.objective
values=result.best.values

#エネルギー関数を最小にする、頂点の取り除き方
print(f"エネルギー関数の最小値 = {objective}")
y_v_values=y_v.evaluate(values)
eliminated=[]
for v in range(N):
  if y_v_values[v]==0:
    eliminated.append(v+1)
print(f"取り除く頂点：{eliminated}")